<a href="https://colab.research.google.com/github/JadhavShrutiS/CSCE580_F25/blob/main/ProjectB_SJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CSCE 580-Project B
by Shruti Jadhav

### First all required packages must be installed and GPU is set to ensure training happens faster (compared to training on CPU)

In [ ]:
# Install required packages (Colab / fresh env)
!pip install -q -U transformers accelerate datasets evaluate scikit-learn matplotlib seaborn pyarrow
!pip install -q -U datasets

In [ ]:
# GPU check (works in Colab after Runtime -> Change runtime type -> GPU)
import torch
if torch.cuda.is_available():
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    print('GPU not available. The notebook will still run on CPU but training is much slower.')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#### First we import the csv data. For easy access to data by all, it is hosted and extracted from a Github repo.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/JadhavShrutiS/CSCE580/refs/heads/main/Project-B/IMDB%20Dataset.csv"
df = pd.read_csv(url)
df.head()

#### Then we use a Stratified Split to split the csv data into train, test and validation data segements. Here 80% data is used to train, 10% is used to validate, and 10% is used for final testing.

In [ ]:
from datasets import Dataset
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

ds = Dataset.from_pandas(df)

reviews = np.array(ds["review"])
sentiments = np.array(ds["sentiment"])
label_map = {"positive": 1, "negative": 0}
labels = np.array([label_map[x] for x in sentiments])

# 80% train, 20% holdout
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
train_idx, hold_idx = next(sss.split(reviews, labels))

hold_labels = labels[hold_idx]
hold_reviews = reviews[hold_idx]

# 50/50 val and test
val_idx, test_idx = train_test_split(
    np.arange(len(hold_reviews)),
    test_size=0.5,
    stratify=hold_labels,
    random_state=50
)

train_ds = Dataset.from_dict({
    "review": reviews[train_idx],
    "labels": labels[train_idx]
})

val_ds = Dataset.from_dict({
    "review": hold_reviews[val_idx],
    "labels": hold_labels[val_idx]
})

test_ds = Dataset.from_dict({
    "review": hold_reviews[test_idx],
    "labels": hold_labels[test_idx]
})


#### Following splitting the data, we processing the data by first tokenizing. We truncate the reviews to a maximum of 256 tokenized text to allow for better handling with the model.

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["review"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

train_tok = train_ds.map(tokenize_fn, batched=True)
val_tok   = val_ds.map(tokenize_fn, batched=True)
test_tok  = test_ds.map(tokenize_fn, batched=True)


#### Then we finish the data processing step by padding the data. This ensures all batches of data being processed have the same length. After the padding is done, the training starts.

#### Classification is configured using binary outputs (num_labels =2), and evaluation metrics are defined using the compute_metrics function.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

def compute_metrics(eval_pred):
    acc = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    prec = evaluate.load("precision")
    rec = evaluate.load("recall")

    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec.compute(predictions=preds, references=labels)["precision"],
        "recall": rec.compute(predictions=preds, references=labels)["recall"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }

import os
os.environ["WANDB_DISABLED"] = "true"

OUT_DIR = "distilbert_model"

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    eval_strategy="epoch",
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#### The training is now started and the training time is recorded.

In [ ]:
import time
start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"Training time (fine-tuned DistilBERT): {training_time:.2f} seconds")

#### After training, the final trained model is saved. This is intially downloaded and then uploaded to huggingface.

#### This ensures that training is only done once. The trained information can be extracted from the saved files.

In [ ]:
FIN_DIR = "distilbert_final"
trainer.save_model(FIN_DIR)
tokenizer.save_pretrained(FIN_DIR)

In [ ]:
import shutil

# shutil.make_archive("distilbert_checkpoints_zip", 'zip', OUT_DIR)
shutil.make_archive("distilbert_final_zip", 'zip', FIN_DIR)

In [ ]:
from google.colab import files
# files.download("distilbert_checkpoints_zip.zip")
files.download("distilbert_final_zip.zip")

#### The following set of code is used to extract the once trained data from fine-tuned DistilBERT. So that even after training once, the rest of the analysis can be run independently.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_path = "ShrutiJadh/Distilbert_finetuned_SJ"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

#### The fine-tuned DistilBERT model is now evaluated on the test set.

In [ ]:
trainer.evaluate(test_tok)

#### The metrics are extracted from the trained data for the fine-tuned DistilBERT case:

In [ ]:
train_eval = trainer.evaluate(train_tok.select(range(300)))
train_accuracy_value = train_eval["eval_accuracy"]


In [ ]:
train_accuracy_list = [(i+1, train_accuracy_value) for i in range(training_args.num_train_epochs)]

#### The graphs for accuracy and loss are then constructed using the extracted metrics.

In [ ]:
import pandas as pd

logs = pd.DataFrame(trainer.state.log_history)

# Keep only entries that have BOTH epoch and eval metrics
eval_acc_df = logs[
    logs["eval_accuracy"].notna() &
    logs["epoch"].notna()
][["epoch", "eval_accuracy"]].drop_duplicates(subset="epoch")

train_loss_df = logs[
    logs["loss"].notna() &
    logs["epoch"].notna()
][["epoch", "loss"]].drop_duplicates(subset="epoch")

eval_loss_df = logs[
    logs["eval_loss"].notna() &
    logs["epoch"].notna()
][["epoch", "eval_loss"]].drop_duplicates(subset="epoch")

eval_acc_df, train_loss_df, eval_loss_df


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.plot(eval_acc_df["epoch"], eval_acc_df["eval_accuracy"], label = "validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy per Epoch")
plt.legend()
plt.grid()
plt.show()

# Loss graph
plt.figure(figsize=(6,4))
plt.plot(train_loss_df["epoch"], train_loss_df["loss"], label="Train Loss")
plt.plot(eval_loss_df["epoch"], eval_loss_df["eval_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()


#### The confusion matrix for fine tuned distilBERT model is constructed as:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

preds_output = trainer.predict(test_tok)
preds = np.argmax(preds_output.predictions, axis=-1)

cm = confusion_matrix(test_ds["labels"], preds)
print(cm)

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_ds["labels"], preds)

plt.figure(figsize=(6,4))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=["Negative", "Positive"],
    yticklabels=["Negative", "Positive"]
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Fine-Tuned DistilBERT")
plt.show()

print(classification_report(test_ds["labels"], preds, target_names=["negative","positive"]))

#### Now the pretrained DistilBERT model without fine-tuning, is used to classify the set

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

pretrained_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)

pretrained_trainer = Trainer(
    model=pretrained_model,
    tokenizer=tokenizer,
    data_collator=data_collator
)

pre_preds_output = pretrained_trainer.predict(test_tok)
pre_preds = np.argmax(pre_preds_output.predictions, axis=-1)

nonft_accuracy = accuracy_score(test_ds["labels"], pre_preds)
nonft_precision, nonft_recall, nonft_f1, _ = precision_recall_fscore_support(
    test_ds["labels"], pre_preds, average="binary"
)

print("\n===== NON-FINE-TUNED DISTILBERT METRICS =====")
print("Accuracy:", nonft_accuracy)
print("Precision:", nonft_precision)
print("Recall:", nonft_recall)
print("F1 Score:", nonft_f1)

print(confusion_matrix(test_ds["labels"], pre_preds))

cm_pre = confusion_matrix(test_ds["labels"], pre_preds)

plt.figure(figsize=(6,4))
sns.heatmap(
    cm_pre,
    annot=True,
    fmt='d',
    cmap='Purples',
    xticklabels=["Negative", "Positive"],
    yticklabels=["Negative", "Positive"]
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Pretrained DistilBERT")
plt.show()
print(classification_report(test_ds["labels"], pre_preds, target_names=["negative","positive"]))

#### Following is the Classical Machine Learning model that uses Support Vector Machine using TF-IDF features

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import time

clf = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=20000)),
    ("svm", LinearSVC())
])
start = time.time()
clf.fit(train_ds["review"], train_ds["labels"])
end = time.time()

print(f"Training time: {end - start:.2f} seconds")
svm_preds = clf.predict(test_ds["review"])

svm_accuracy = accuracy_score(test_ds["labels"], svm_preds)
svm_precision, svm_recall, svm_f1, _ = precision_recall_fscore_support(
    test_ds["labels"], svm_preds, average="binary"
)

print("\n===== SVM METRICS =====")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("F1 Score:", svm_f1)

# print(confusion_matrix(test_ds["labels"], svm_preds))
cm_svm = confusion_matrix(test_ds["labels"], svm_preds)

plt.figure(figsize=(6,4))
sns.heatmap(
    cm_svm,
    annot=True,
    fmt='d',
    cmap='Greens',
    xticklabels=["Negative", "Positive"],
    yticklabels=["Negative", "Positive"]
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - SVM Classifier")
plt.show()

print(classification_report(test_ds["labels"], svm_preds, target_names=["negative","positive"]))

#### Lastly, functions are written to predict any user provided review.

In [ ]:
model.to(device)
model.eval()

def predict_finetuned(text):
    import time
    start = time.time()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    # move inputs to the SAME device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()
    end = time.time()
    print(f"Inference time: {end -start:.4f} seconds")

    return "positive" if pred == 1 else "negative"

In [ ]:
pretrained_model.to(device)
pretrained_model.eval()

def predict_pretrained(text):
    import time
    start = time.time()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    # move inputs to the SAME device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()
    end = time.time()
    print(f"Inference time: {end -start:.4f} seconds")
    return "positive" if pred == 1 else "negative"

In [ ]:
def predict_svm(text):
    import time
    start = time.time()
    pred = clf.predict([text])[0]
    end = time.time()
    print(f"Inference time: {end -start:.4f} seconds")
    return "positive" if pred == 1 else "negative"

## Now let's predict sentiment.

#### First we use the TestCase 1- Review level: Easy.



In [ ]:
predict_finetuned("While I hoped I would be wrong, I anticipated not being a fan of Wicked For Good…unfortunately, despite Ariana Grande and Cynthia Erivo singing their hearts out, it was a bit of a mess.")


In [ ]:
predict_pretrained("While I hoped I would be wrong, I anticipated not being a fan of Wicked For Good…unfortunately, despite Ariana Grande and Cynthia Erivo singing their hearts out, it was a bit of a mess.")


In [ ]:
predict_svm("While I hoped I would be wrong, I anticipated not being a fan of Wicked For Good…unfortunately, despite Ariana Grande and Cynthia Erivo singing their hearts out, it was a bit of a mess.")


Review/Prompt: "While I hoped I would be wrong, I anticipated not being a fan of Wicked For Good…unfortunately, despite Ariana Grande and Cynthia Erivo singing their hearts out, it was a bit of a mess."

GPT response: negative

Simulated Inference time: 8 ms

#### Lets move to TestCase 2- Review complexity level: Medium





In [ ]:
predict_finetuned("There's no way to make this documentary sound like the upbeat, rousing and often downright hilarious romp it is, but here goes: At the urging of comedian Tig Notaro, poet and spoken-word star Andrea Gibson and life partner and fellow poet Megan Falley invited filmmaker Ryan White and his crew into their home in 2021. It was mid-pandemic, and the crew was allowed full access to the couple's every thought and action as they dealt with turtledove love, mailbox madness, and, here's the part where you say, no, this does not sound like a good time ,Gibson's Stage 4 ovarian cancer journey. At the Middleburg Film Festival screening I attended in October, three months after Gibson's death, the director spoke beforehand, giving the audience permission to laugh, which it definitely did. It also sniffled a bit, but less than you might expect, because Gibson's vibrant, assertively affirmative outlook doesn't really brook tears, and the filmmaker's warmth and humor, even in times of despair, gives the story a radiance that makes mundane moments feel precious, while allowing hopeful moments to raise goosebumps.")

In [ ]:
predict_pretrained("There's no way to make this documentary sound like the upbeat, rousing and often downright hilarious romp it is, but here goes: At the urging of comedian Tig Notaro, poet and spoken-word star Andrea Gibson and life partner and fellow poet Megan Falley invited filmmaker Ryan White and his crew into their home in 2021. It was mid-pandemic, and the crew was allowed full access to the couple's every thought and action as they dealt with turtledove love, mailbox madness, and, here's the part where you say, no, this does not sound like a good time ,Gibson's Stage 4 ovarian cancer journey. At the Middleburg Film Festival screening I attended in October, three months after Gibson's death, the director spoke beforehand, giving the audience permission to laugh, which it definitely did. It also sniffled a bit, but less than you might expect, because Gibson's vibrant, assertively affirmative outlook doesn't really brook tears, and the filmmaker's warmth and humor, even in times of despair, gives the story a radiance that makes mundane moments feel precious, while allowing hopeful moments to raise goosebumps.")

In [ ]:
predict_svm("There's no way to make this documentary sound like the upbeat, rousing and often downright hilarious romp it is, but here goes: At the urging of comedian Tig Notaro, poet and spoken-word star Andrea Gibson and life partner and fellow poet Megan Falley invited filmmaker Ryan White and his crew into their home in 2021. It was mid-pandemic, and the crew was allowed full access to the couple's every thought and action as they dealt with turtledove love, mailbox madness, and, here's the part where you say, no, this does not sound like a good time ,Gibson's Stage 4 ovarian cancer journey. At the Middleburg Film Festival screening I attended in October, three months after Gibson's death, the director spoke beforehand, giving the audience permission to laugh, which it definitely did. It also sniffled a bit, but less than you might expect, because Gibson's vibrant, assertively affirmative outlook doesn't really brook tears, and the filmmaker's warmth and humor, even in times of despair, gives the story a radiance that makes mundane moments feel precious, while allowing hopeful moments to raise goosebumps.")

Review/Prompt: "There's no way to make this documentary sound like the upbeat, rousing and often downright hilarious romp it is, but here goes: At the urging of comedian Tig Notaro, poet and spoken-word star Andrea Gibson and life partner and fellow poet Megan Falley invited filmmaker Ryan White and his crew into their home in 2021. It was mid-pandemic, and the crew was allowed full access to the couple's every thought and action as they dealt with turtledove love, mailbox madness, and, here's the part where you say, no, this does not sound like a good time ,Gibson's Stage 4 ovarian cancer journey. At the Middleburg Film Festival screening I attended in October, three months after Gibson's death, the director spoke beforehand, giving the audience permission to laugh, which it definitely did. It also sniffled a bit, but less than you might expect, because Gibson's vibrant, assertively affirmative outlook doesn't really brook tears, and the filmmaker's warmth and humor, even in times of despair, gives the story a radiance that makes mundane moments feel precious, while allowing hopeful moments to raise goosebumps."

GPT Response: Positive

Simulated Inference time: 14 ms

#### Finally TestCase 3- Review complexity level: High


In [ ]:
predict_finetuned("Cannot understand the 1 out of 10 harsh review. 1 means its the worst film you have ever seen which this is not. Presumably its either because they were  offended by the film being directed, written and acted by a woman who was married to Elon Musk and is giving it the silver spoon brush off or either offended at a Scottish movie with mostly English actors in it, who knows but the film is worth a watch with your partner as it is most definitely not just a womans movie as I quite enjoyed it and am a bit of an old romantic at heart myself.Its not going to win any oscars but it has some moments of comedy magic in that typical Glasgow way which only us Weegies can fully appreciate. Not a bad effort at all from Talulah Riley at all, its just a shame that people want to bash her because she is the ex of Elon Musk which is a shame as she certainly has acting talent that is for sure and for her first directorial debut it was not half bad")

In [ ]:
predict_pretrained("Cannot understand the 1 out of 10 harsh review. 1 means its the worst film you have ever seen which this is not. Presumably its either because they were  offended by the film being directed, written and acted by a woman who was married to Elon Musk and is giving it the silver spoon brush off or either offended at a Scottish movie with mostly English actors in it, who knows but the film is worth a watch with your partner as it is most definitely not just a womans movie as I quite enjoyed it and am a bit of an old romantic at heart myself.Its not going to win any oscars but it has some moments of comedy magic in that typical Glasgow way which only us Weegies can fully appreciate. Not a bad effort at all from Talulah Riley at all, its just a shame that people want to bash her because she is the ex of Elon Musk which is a shame as she certainly has acting talent that is for sure and for her first directorial debut it was not half bad")

In [ ]:
predict_svm("Cannot understand the 1 out of 10 harsh review. Presumably its either because they were  offended by the film being directed, written and acted by a woman who was married to Elon Musk and is giving it the silver spoon brush off or either offended at a Scottish movie with mostly English actors in it, who knows but the film is worth a watch with your partner as it is most definitely not just a womans movie as I quite enjoyed it and am a bit of an old romantic at heart myself.Its not going to win any oscars but it has some moments of comedy magic in that typical Glasgow way which only us Weegies can fully appreciate. Not a bad effort at all from Talulah Riley at all, its just a shame that people want to bash her because she is the ex of Elon Musk which is a shame as she certainly has acting talent that is for sure and for her first directorial debut it was not half bad")

Review prompt: "Cannot understand the 1 out of 10 harsh review. 1 means its the worst film you have ever seen which this is not. Presumably its either because they were  offended by the film being directed, written and acted by a woman who was married to Elon Musk and is giving it the silver spoon brush off or either offended at a Scottish movie with mostly English actors in it, who knows but the film is worth a watch with your partner as it is most definitely not just a womans movie as I quite enjoyed it and am a bit of an old romantic at heart myself.

Its not going to win any oscars but it has some moments of comedy magic in that typical Glasgow way which only us Weegies can fully appreciate. Not a bad effort at all from Talulah Riley at all, its just a shame that people want to bash her because she is the ex of Elon Musk which is a shame as she certainly has acting talent that is for sure and for her first directorial debut it was not half bad!"

GPT Response: Positive

Simulated Inference time: 11ms